# 뉴스 크롤링

## 제목

In [ ]:
# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver


# Install selenium
pip install selenium

In [ ]:
# 셀레니움 module import 
# from selenium import webdriver
# 2023.05.08 selenium webdriver 사용방법 변경 
from selenium.webdriver.common.by import By
from selenium import webdriver
import time 
from time import sleep
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import requests
import re
import os

# Dataframe 
import pandas as pd
import numpy as np

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('disable-dev-shm-usage')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
driver = webdriver.Chrome('chromedriver', options=chrome_options)

In [ ]:
train_df=pd.read_csv('../data/train.csv')

In [ ]:
train_df

,ID,text,target,url,date
0,ynat-v1_train_00000,유튜브 내달 2일까지 크리에이터 지원 공간 운영,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.30. 오전 10:36
1,ynat-v1_train_00001,어버이날 막따가 흐려저…남부지방 여튼 황사,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.08. 오전 5:25
2,ynat-v1_train_00002,내년부터 국가RD 평가 때 논문건수는 반영 않는다,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.15. 오후 12:00
3,ynat-v1_train_00003,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.02.28. 오전 9:54
4,ynat-v1_train_00004,회새긴간 작까 김동시 걍심꼬백 뜽 새 소설집 뚜권 출간,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.04.03. 오전 7:05
...,...,...,...,...,...
45673,ynat-v1_train_45673,사우디 아람코 주요 석유시설·유전 드론 공격에 화재,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.09.14. 오후 4:36
45674,ynat-v1_train_45674,박기원 감독 눈치 보지 말고…비예나 눈치 본 건 아닌데,5,https://sports.news.naver.com/news.nhn?oid=001...,2019.11.03 17:06
45675,ynat-v1_train_45675,아시안게임 첫 경기 앞둔 라건아 인도네시아 팬이라도 ...,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.08.13 17:54
45676,ynat-v1_train_45676,트럼프 미중 무역협상 1단계 합의 서명식 가질 것종합,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.25. 오후 4:36


In [ ]:
len(train_df['url'].unique())

45678

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

def get_title(URL):
  try: 
    original_html = requests.get(URL, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    # print(html)

    # 뉴스 제목 가져오기
    title = html.select("div#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=title)
    return title[1:-1]
  except:
    return 'No_Title'

def get_title1(URL):
  try: 
    original_html = requests.get(URL, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    # print(html)

    # 뉴스 제목 가져오기
    title = html.select("div#content > div > div.content > div > div.news_headline > h4")
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=title)
    return title[1:-1]
  except:
    return 'No_Title'

def get_title2(URL):
  try: 
    original_html = requests.get(URL, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    # print(html)

    # 뉴스 제목 가져오기
    title = html.select("div#ct > div.media_end_head.go_trans > div.media_end_head_title")
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=title)
    return title[1:-1].strip()
  except:
    return 'No_Title'

In [ ]:
for i in tqdm(range(len(train_df))):
  train_df.loc[i,'crawling_title']=get_title(train_df.loc[i,'url'])

In [ ]:
df1=train_df[train_df['crawling_title']!=''].reset_index(drop=True)
df2=train_df[train_df['crawling_title']==''].reset_index(drop=True)

In [ ]:
for i in tqdm(range(len(df2))):
  df2.loc[i,'crawling_title']=get_title1(df2.loc[i,'url'])

In [ ]:
data=pd.concat([df1,df2]).sort_values('ID').reset_index(drop=True)

In [ ]:
final1=data[data['crawling_title']!=''].reset_index(drop=True)
final2=data[data['crawling_title']==''].reset_index(drop=True)

In [ ]:
for i in tqdm(range(len(final2))):
  final2.loc[i,'crawling_title']=get_title2(final2.loc[i,'url'])

In [ ]:
final=pd.concat([final1,final2]).sort_values('ID').reset_index(drop=True)

## 본문

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.

def get_content(URL):
  try:
    original_html = requests.get(URL, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    # print(html)
    # 뉴스 본문 가져오기
    content = html.select("div#dic_area") 
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))
    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')
    return content[1:-1].strip()
  except:
    return 'No_Content'

def get_title1(URL):
  try: 
    original_html = requests.get(URL, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    # print(html)

    # 뉴스 제목 가져오기
    title = html.select("div#content > div > div.content > div > div.news_headline > h4")
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=title)
    return title[1:-1]
  except:
    return 'No_Title'

def get_title2(URL):
  try: 
    original_html = requests.get(URL, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    # print(html)

    # 뉴스 제목 가져오기
    title = html.select("div#ct > div.media_end_head.go_trans > div.media_end_head_title")
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=title)
    return title[1:-1].strip()
  except:
    return 'No_Title'

In [ ]:
for i in tqdm(range(len(train_df))):
  train_df.loc[i,'crawling_content']=get_content(train_df.loc[i,'url'])

In [ ]:
df1=train_df[train_df['crawling_content']!=''].reset_index(drop=True)
df2=train_df[train_df['crawling_content']==''].reset_index(drop=True)

In [ ]:
for i in tqdm(range(len(df2))):
  df2.loc[i,'crawling_content']=get_content1(df2.loc[i,'url'])

In [ ]:
data=pd.concat([df1,df2]).sort_values('ID').reset_index(drop=True)

In [ ]:
final1=data[data['crawling_content']!=''].reset_index(drop=True)
final2=data[data['crawling_content']==''].reset_index(drop=True)

In [ ]:
final=pd.concat([final1,final2]).sort_values('ID').reset_index(drop=True)

In [ ]:
final.to_csv('../data/train_title_content_crawling.csv',encoding='utf-8-sig',index=False)